In [1]:
import requests
from requests import Session

from base64 import b64encode, b64decode
import hashlib
import sys
import crypto
sys.modules['Crypto'] = crypto
sys.path.append(r'/c/Users/Go/Desktop/TapoP100-main/PyP100/')


from Crypto.PublicKey import RSA
import time
import json
from Crypto.Cipher import AES, PKCS1_OAEP, PKCS1_v1_5
import tp_link_cipher
import ast
import pkgutil
import uuid
import json

ERROR_CODES = {
	"0": "Success",
	"-1010": "Invalid Public Key Length",
	"-1012": "Invalid terminalUUID",
	"-1501": "Invalid Request or Credentials",
	"1002": "Incorrect Request",
	"-1003": "JSON formatting error "
}

In [2]:
import requests
from requests import Session

from base64 import b64encode, b64decode
import hashlib
import sys
import crypto
sys.modules['Crypto'] = crypto
sys.path.append(r'/c/Users/Go/Desktop/TapoP100-main/PyP100/')


from Crypto.PublicKey import RSA
import time
import json
from Crypto.Cipher import AES, PKCS1_OAEP, PKCS1_v1_5
import tp_link_cipher
import ast
import pkgutil
import uuid
import json


#Old Functions to get device list from tplinkcloud
def getToken(email, password):
	URL = "https://eu-wap.tplinkcloud.com"
	Payload = {
		"method": "login",
		"params": {
			"appType": "Tapo_Ios",
			"cloudUserName": email,
			"cloudPassword": password,
			"terminalUUID": "0A950402-7224-46EB-A450-7362CDB902A2"
		}
	}

	return requests.post(URL, json=Payload).json()['result']['token']


def getDeviceList(email, password):
	URL = "https://eu-wap.tplinkcloud.com?token=" + getToken(email, password)
	Payload = {
		"method": "getDeviceList",
	}

	return requests.post(URL, json=Payload).json()

ERROR_CODES = {
	"0": "Success",
	"-1010": "Invalid Public Key Length",
	"-1012": "Invalid terminalUUID",
	"-1501": "Invalid Request or Credentials",
	"1002": "Incorrect Request",
	"-1003": "JSON formatting error "
}

class P100():
	def __init__ (self, ipAddress, email, password):
		self.ipAddress = ipAddress
		self.terminalUUID = str(uuid.uuid4())

		self.email = email
		self.password = password
		self.session = Session()

		self.errorCodes = ERROR_CODES

		self.encryptCredentials(email, password)
		self.createKeyPair()

	def encryptCredentials(self, email, password):
		#Password Encoding
		self.encodedPassword = tp_link_cipher.TpLinkCipher.mime_encoder(password.encode("utf-8"))

		#Email Encoding
		self.encodedEmail = self.sha_digest_username(email)
		self.encodedEmail = tp_link_cipher.TpLinkCipher.mime_encoder(self.encodedEmail.encode("utf-8"))

	def createKeyPair(self):
		self.keys = RSA.generate(1024)

		self.privateKey = self.keys.exportKey("PEM")
		self.publicKey  = self.keys.publickey().exportKey("PEM")

	def decode_handshake_key(self, key):
		decode: bytes = b64decode(key.encode("UTF-8"))
		decode2: bytes = self.privateKey

		cipher = PKCS1_v1_5.new(RSA.importKey(decode2))
		do_final = cipher.decrypt(decode, None)
		if do_final is None:
			raise ValueError("Decryption failed!")

		b_arr:bytearray = bytearray()
		b_arr2:bytearray = bytearray()

		for i in range(0, 16):
			b_arr.insert(i, do_final[i])
		for i in range(0, 16):
			b_arr2.insert(i, do_final[i + 16])

		return tp_link_cipher.TpLinkCipher(b_arr, b_arr2)

	def sha_digest_username(self, data):
		b_arr = data.encode("UTF-8")
		digest = hashlib.sha1(b_arr).digest()

		sb = ""
		for i in range(0, len(digest)):
			b = digest[i]
			hex_string = hex(b & 255).replace("0x", "")
			if len(hex_string) == 1:
				sb += "0"
				sb += hex_string
			else:
				sb += hex_string

		return sb

	def handshake(self):
		URL = f"http://{self.ipAddress}/app"
		Payload = {
			"method":"handshake",
			"params":{
				"key": self.publicKey.decode("utf-8"),
				"requestTimeMils": int(round(time.time() * 1000))
			}
		}

		r = self.session.post(URL, json=Payload, timeout=2)

		encryptedKey = r.json()["result"]["key"]
		self.tpLinkCipher = self.decode_handshake_key(encryptedKey)

		try:
			self.cookie = r.headers["Set-Cookie"][:-13]

		except:
			errorCode = r.json()["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def login(self):
		URL = f"http://{self.ipAddress}/app"
		Payload = {
			"method":"login_device",
			"params":{
				"username": self.encodedEmail,
				"password": self.encodedPassword
			},
			"requestTimeMils": int(round(time.time() * 1000)),
		}
		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method":"securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		try:
			self.token = ast.literal_eval(decryptedResponse)["result"]["token"]
		except:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOn(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "set_device_info",
			"params":{
				"device_on": True
			},
			"requestTimeMils": int(round(time.time() * 1000)),
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOff(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "set_device_info",
			"params":{
				"device_on": False
			},
			"requestTimeMils": int(round(time.time() * 1000)),
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers, timeout=2)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def getDeviceInfo(self):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "get_device_info",
			"requestTimeMils": int(round(time.time() * 1000)),
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method":"securePassthrough",
			"params":{
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)
		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		return json.loads(decryptedResponse)

	def getDeviceName(self):
		self.handshake()
		self.login()
		data = self.getDeviceInfo()

		if data["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")
		else:
			encodedName = data["result"]["nickname"]
			name = b64decode(encodedName)
			return name.decode("utf-8")

	def turnOnWithDelay(self, delay):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "add_countdown_rule",
			"params": {
				"delay": int(delay),
				"desired_states": {
					"on": True
				},
				"enable": True,
				"remain": int(delay)
			},
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")

	def turnOffWithDelay(self, delay):
		URL = f"http://{self.ipAddress}/app?token={self.token}"
		Payload = {
			"method": "add_countdown_rule",
			"params": {
				"delay": int(delay),
				"desired_states": {
					"on": False
				},
				"enable": True,
				"remain": int(delay)
			},
			"terminalUUID": self.terminalUUID
		}

		headers = {
			"Cookie": self.cookie
		}

		EncryptedPayload = self.tpLinkCipher.encrypt(json.dumps(Payload))

		SecurePassthroughPayload = {
			"method": "securePassthrough",
			"params": {
				"request": EncryptedPayload
			}
		}

		r = self.session.post(URL, json=SecurePassthroughPayload, headers=headers)

		decryptedResponse = self.tpLinkCipher.decrypt(r.json()["result"]["response"])

		if ast.literal_eval(decryptedResponse)["error_code"] != 0:
			errorCode = ast.literal_eval(decryptedResponse)["error_code"]
			errorMessage = self.errorCodes[str(errorCode)]
			raise Exception(f"Error Code: {errorCode}, {errorMessage}")


In [5]:
p_controller = P100('221.151.78.120:80','rhqudrjs2@naver.com','qudrjs12#')
p_controller.handshake()
p_controller.login()

In [29]:
getToken('rhqudrjs2@naver.com','qudrjs12#')

'bd3eedca-BTx0fIKRX2CakiFJ8rnRbDX'

In [28]:
getDeviceList('rhqudrjs2@naver.com','qudrjs12#')

{'error_code': 0,
 'result': {'deviceList': [{'deviceType': 'SMART.TAPOPLUG',
    'role': 0,
    'fwVer': '1.4.10 Build 20211104 Rel. 35882',
    'appServerUrl': 'https://aps1-wap.tplinkcloud.com',
    'deviceRegion': 'ap-southeast-1',
    'deviceId': '80222F3E37C0574D5D4412E6AAF58DAD1F3FF970',
    'deviceName': 'P100',
    'deviceHwVer': '1.20.0',
    'alias': 'NjAzLXRlc3Q=',
    'deviceMac': '5CA6E6C13C79',
    'oemId': 'D7C5ADFD37188938BA82FD0FE208D1E7',
    'deviceModel': 'P100',
    'hwId': '9994A0A7D5B29645B8150C392284029D',
    'fwId': '1D18AD293A25ABDE41405B20C6F98816',
    'isSameRegion': False,
    'status': 0},
   {'deviceType': 'SMART.TAPOPLUG',
    'role': 0,
    'fwVer': '1.4.10 Build 20211104 Rel. 35882',
    'appServerUrl': 'https://aps1-wap.tplinkcloud.com',
    'deviceRegion': 'ap-southeast-1',
    'deviceId': '8022B77263924A0418F2500A15D2A6D61F3F5DBF',
    'deviceName': 'P100',
    'deviceHwVer': '1.20.0',
    'alias': 'NjAzX+qzteq4sOyInO2ZmOq4sF8x',
    'deviceMac':

In [52]:
testP100 = P100('221.151.78.120:82','rhqudrjs2@naver.com','qudrjs12#')
testP100.handshake()
testP100.login()

In [55]:
testP100.turnOn()
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': True,
  'on_time': 0,
  'overheated': False,
  'nickname': 'NjAzX+qzteq4sOyyreygleq4sF8y',
  'location': 'office',
  'avatar': 'bread_maker',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.15',
  'ssid': 'aXB0aW1lXzYwMw==',
  'signal_level': 1,
  'rssi': -77,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': '',
  'default_states': {'type': 'last_states', 'state': {}}}}

In [14]:
testP100.turnOff()
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': False,
  'on_time': 0,
  'overheated': False,
  'nickname': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
  'location': 'office',
  'avatar': 'plug',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.115',
  'ssid': 'RUhSTkNfRFNfMkc=',
  'signal_level': 3,
  'rssi': -49,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': 'ko_KR',
  'default_states': {'type': 'last_states', 'state': {}}}}

In [16]:
testP100.turnOn()
testP100.getDeviceInfo()

{'error_code': 0,
 'result': {'device_id': '80222EAB5A5A857DC2811934D688F8E71F30DF6F',
  'fw_ver': '1.4.10 Build 20211104 Rel. 35882',
  'hw_ver': '1.20.0',
  'type': 'SMART.TAPOPLUG',
  'model': 'P100',
  'mac': '5C-A6-E6-3A-B8-03',
  'hw_id': '9994A0A7D5B29645B8150C392284029D',
  'fw_id': '1D18AD293A25ABDE41405B20C6F98816',
  'oem_id': 'D7C5ADFD37188938BA82FD0FE208D1E7',
  'specs': 'EU',
  'device_on': True,
  'on_time': 5,
  'overheated': False,
  'nickname': '7Iqk66eI7Yq4IO2UjOufrOq3uCB0ZXN0',
  'location': 'office',
  'avatar': 'plug',
  'longitude': -1879048193,
  'latitude': -1879048193,
  'has_set_location_info': False,
  'ip': '192.168.0.115',
  'ssid': 'RUhSTkNfRFNfMkc=',
  'signal_level': 3,
  'rssi': -48,
  'region': 'Asia/Seoul',
  'time_diff': 540,
  'lang': 'ko_KR',
  'default_states': {'type': 'last_states', 'state': {}}}}

In [18]:
testP100.getDeviceInfo()['result']['on_time']

42

In [20]:
print('1')

1


In [21]:
import pandas as pd

In [24]:
data = {
    'id':['ehrnc'],
    'pw':['64581'],
}
id_data = pd.DataFrame(data)

In [25]:
id_data.to_csv()


,id,pw
0,ehrnc,64581


In [29]:
import os
from glob import glob

path = os.getcwd()
os.path.join(path,'TapoP100',"DB")
id_data.to_csv(os.path.join(path,'TapoP100',"DB","user_data.csv"))

In [79]:
import os
from glob import glob

path = os.getcwd()

# data = {
#         'ip':[],
#         'user_id':[],
#         'plug_name':[],
#         'on_state':[],
#     }

# plug_data = pd.DataFrame(data)
# plug_data.to_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"),index = False)
tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"))
sample_data = {
        'ip':'192.168.0.115',
        'user_id':'ehrnc',
        'plug_name':'test_2',
        'on_state':'false',
    }
tt = tt.append(sample_data,ignore_index=True)
tt.to_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"),index = False)

In [84]:
pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"))

,ip,user_id,plug_name,on_state
0,192.168.0.118,ehrnc,test_1,False


In [75]:
from flask import Blueprint, stream_with_context, request, Response, jsonify
jsonify(tt)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [83]:
path = os.getcwd()
tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data",'ehrnc'+'.csv',))
tt

,ip,user_id,plug_name,on_state
0,192.168.0.118,ehrnc,test_1,False


In [92]:
len(tt.ip)

sample_data = {
        'ip':'192.168.0.118',
        'user_id':'ehrnc',
        'plug_name':'test_2',
        'on_state':'false',
    }
tt.append(sample_data,ignore_index=True).ip



set(tt.append(sample_data,ignore_index=True).ip)

{'192.168.0.118'}

In [93]:
set(tt.ip)

{'192.168.0.118'}

In [94]:
set(tt.ip) == set(tt.append(sample_data,ignore_index=True).ip)

True

In [91]:
tt.append(sample_data,ignore_index=True).ip

0    192.168.0.118
1    192.168.0.118
Name: ip, dtype: object

In [89]:
tt.ip.values

array(['192.168.0.118'], dtype=object)

In [82]:
p_controller = P100('192.168.0.118','rhqudrjs2@naver.com','qudrjs12#')
p_controller.handshake()
p_controller.login()

p_controller.getDeviceInfo()['result']['on_time']

0

In [104]:
ip = '192.168.0.118'
user_id = 'ehrnc'
email = 'rhqudrjs2@naver.com'
pw = 'qudrjs12#'



p_controller = P100(ip,email,pw)
p_controller.handshake()
p_controller.login()
p_controller.turnOn()


path = os.getcwd()
tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data",'ehrnc'+'.csv',))
tt.loc[tt.ip == ip,['on_state']] = False
tt

,ip,user_id,plug_name,on_state
0,192.168.0.118,ehrnc,gfgfgf,False


In [107]:
ip = '192.168.0.118'
user_id = 'ehrnc'
email = 'rhqudrjs2@naver.com'
pw = 'qudrjs12#'



p_controller = P100(ip,email,pw)
p_controller.handshake()
p_controller.login()
p_controller.turnOff()


path = os.getcwd()
tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data",'ehrnc'+'.csv',))
tt.loc[tt.ip == ip,['on_state']] = True
tt

,ip,user_id,plug_name,on_state
0,192.168.0.118,ehrnc,gfgfgf,False


In [111]:
tt.loc[tt.ip == ip,['on_state']] = False

In [112]:
tt

,ip,user_id,plug_name,on_state
0,192.168.0.118,ehrnc,gfgfgf,False


In [5]:
ip = '192.168.0.12'
user_id = 'ehrnc'
email = 'rhqudrjs2@naver.com'
pw = 'qudrjs12#'



p_controller = P100(ip,email,pw)
p_controller.handshake()
p_controller.login()
p_controller.turnOn()


# path = os.getcwd()
# tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data",'ehrnc'+'.csv',))
# tt.loc[tt.ip == ip] = True
# tt

ConnectTimeout: HTTPConnectionPool(host='192.168.0.12', port=80): Max retries exceeded with url: /app (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x0000021DC2225748>, 'Connection to 192.168.0.12 timed out. (connect timeout=2)'))

In [ ]:
data = {
    'id':['ehrnc'],
    'pw':['64581'],
}
id_data = pd.DataFrame(data)

import os
from glob import glob

path = os.getcwd()

# data = {
#         'ip':[],
#         'user_id':[],
#         'plug_name':[],
#         'on_state':[],
#     }


tt = pd.read_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"))
sample_data = {
        'ip':'192.168.0.115',
        'user_id':'ehrnc',
        'plug_name':'test_2',
        'on_state':'false',
    }
tt = tt.append(sample_data,ignore_index=True)
tt.to_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc.csv"),index = False)

In [113]:
sample_data = {
        'time':[],
        'ip':[],
        'state':[],
    }

dd = pd.DataFrame(sample_data)



history_data = pd.DataFrame(dd)
history_data.to_csv(os.path.join(path,'TapoP100',"DB","controller_data","ehrnc_history.csv"),index = False)

In [114]:
time.time()

1652171979.6683047

In [118]:
from datetime import datetime

str(datetime.timedelta(seconds=time.time()))

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [121]:
sample_data = {
        'time':[],
        'ip':[],
        'state':[],
    }

dd = pd.DataFrame(sample_data)
sam = {'time': '2022-05-10 17:52:41', 'ip': '192.168.0.118', 'state': 'off'}
dd.append(sam,ignore_index=True)

,time,ip,state
0,2022-05-10 17:52:41,192.168.0.118,off


In [ ]:
# 사용방법
import schedule
import time
 
def job():
    print("I'm working...")
 

# 10초에 한번씩 실행
schedule.every(10).seconds.do(job)
# 10분에 한번씩 실행
schedule.every(10).minutes.do(job)
# 매 시간 실행
schedule.every().hour.do(job)
# 매일 10:30 에 실행
schedule.every().day.at("10:30").do(job)
# 매주 월요일 실행
schedule.every().monday.do(job)
# 매주 수요일 13:15 에 실행
schedule.every().wednesday.at("13:15").do(job)
 

while True:
    schedule.run_pending()
    time.sleep(1)

In [180]:
schedule.run_pending()

I'm working...


In [183]:
import pandas
#pip install pymysql 
import pymysql

In [217]:
#데이터 베이스와 연결하는 부분
#mysql은 pymysql.connect를 사용하여 데이터베이스와 연결 
#user=mysql 데이터베이스 사용자 이름 , 
#passwd=mysql 데이터 베이스 비밀번호 
#host= mysql 데이터베이스 주소 
#port=mysql데이터 베이스 포트 
#db= 접속하려는 데이터베이스 

conn = pymysql.connect(
user='room_test',
passwd='ehrnc64581',
# 222.108.71.247 (외부접속)
# 192.168.0.42
host='222.108.71.247',
port=3306,
db='testdb',
)

#TEST
sql="SELECT * FROM data WHERE 센서번호=1 AND 날짜 > now() - INTERVAL 1 HOUR;"

#5분데이터 불러오는 부분
# sql="SELECT * FROM data WHERE 날짜 > now() - INTERVAL 5 MINUTE;"

#10분데이터 불러오는 부분
# sql="SELECT * FROM myTable WHERE date > now() - INTERVAL 10 MINUTE;"

In [218]:
df=pandas.read_sql_query(sql,conn)
df

,id,날짜,센서번호,온도,습도,PM,CO2,TVOC
0,24470,2022-05-11 16:31:21,1,26.667,20.131,5.53,665.542,137.8
1,24472,2022-05-11 16:31:31,1,26.691,20.126,5.565,664.932,141.4
2,24475,2022-05-11 16:31:41,1,26.683,20.137,5.225,663.895,141.6
3,24477,2022-05-11 16:31:51,1,26.667,20.127,5.328,663.678,142.6
4,24479,2022-05-11 16:32:01,1,26.67,20.084,5.46,662.967,139.4
...,...,...,...,...,...,...,...,...
355,25290,2022-05-11 17:30:31,1,26.595,21.518,3.836,761.143,131.4
356,25292,2022-05-11 17:30:41,1,26.59,21.462,3.871,760.709,125.6
357,25294,2022-05-11 17:30:51,1,26.622,21.405,3.983,759.824,131.2
358,25297,2022-05-11 17:31:01,1,26.609,21.472,4.009,758.563,126.8


In [201]:
float(df.loc[:,['CO2']].values[-1][0])

515.969

In [ ]:
# http://222.108.71.247:51213/off?ip=${ip}&user_id=${user_id}

In [209]:
ip = '192.168.0.118'
email = 'rhqudrjs2@naver.com'
pw = 'qudrjs12#'

# conn = pymysql.connect(
# user='room_test',
# passwd='ehrnc64581',
# # 222.108.71.247 (외부접속)
# # 192.168.0.42
# host='222.108.71.247',
# port=3306,
# db='testdb',
# )

#TEST
# sql="SELECT * FROM data WHERE 센서번호=01 AND 날짜 > now() - INTERVAL 1 HOUR;"

#5분데이터 불러오는 부분
sql="SELECT * FROM data WHERE 센서번호=01 AND 날짜 > now() - INTERVAL 5 MINUTE;"




p_controller = P100(ip,email,pw)
p_controller.handshake()
p_controller.login()
# p_controller.turnOn()

while True:
    time.sleep(10)
    
    #
    conn = pymysql.connect(
    user='room_test',
    passwd='ehrnc64581',
    # 222.108.71.247 (외부접속)
    # 192.168.0.42
    host='222.108.71.247',
    port=3306,
    db='testdb',
    )
    
    #
    df=pandas.read_sql_query(sql,conn)
    conn.close()
    
    #
    co2_val = float(df.loc[:,['CO2']].values[-1][0])
    print('현재 co2: ',co2_val)
    if co2_val > 700:
        p_controller.turnOn()
    else:
        p_controller.turnOff()





    

현재 co2:  548.691
현재 co2:  549.803
현재 co2:  546.546
현재 co2:  554.68
현재 co2:  749.518
현재 co2:  740.472
현재 co2:  744.304
현재 co2:  756.776
현재 co2:  748.302
현재 co2:  740.041
현재 co2:  735.287
현재 co2:  734.161
현재 co2:  729.862
현재 co2:  713.993
현재 co2:  614.264
현재 co2:  596.709
현재 co2:  595.14
현재 co2:  640.585
현재 co2:  682.288
현재 co2:  649.994
현재 co2:  704.777
현재 co2:  924.969
현재 co2:  903.476
현재 co2:  849.94
현재 co2:  726.705
현재 co2:  725.49
현재 co2:  716.073
현재 co2:  706.998
현재 co2:  674.959
현재 co2:  627.467
현재 co2:  603.61


KeyboardInterrupt: 